# Python Client `s2orc-doc2json`

In [1]:
import os, re, ipdb, traceback, sys

In [2]:
list_pdfs = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf") 
list_df_tei = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_tei_xml") 
list_pdf_json = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_json") 
list_pdf_txt = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_txt") 

In [3]:
print(f"Lenght pdf considered: {len(list_pdfs)}\nLenght python grobid tei.xml: {len(list_df_tei)}\
        \nLenght python grobid json: {len(list_pdfs)}\nLenght python grobid txt: {len(list_pdfs)}")

Lenght pdf considered: 5203
Lenght python grobid tei.xml: 5194        
Lenght python grobid json: 5203
Lenght python grobid txt: 5203


In [4]:
list_latex_src = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/exp/arxiv_src") 
list_latex_src_convert = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/exp/arxiv_src_txt") 

In [5]:
print(f"Lenght pdf with latex source code: {len(list_latex_src)}\
        \nLenght of files that could be converted from .tex to .txt using pandoc (locally): {len(list_latex_src_convert)}")

Lenght pdf with latex source code: 5195        
Lenght of files that could be converted from .tex to .txt using pandoc (locally): 3574


## Evaluate TDMS in the parsed texts 

### New Pipiline stats

In [6]:
def get_stats_from_files(path_to_annotation="resultsAnnotation.tsv", 
                         file_extention=".txt",
                         path_to_txt_folder="../pdf_txt/"):
    with open(path_to_annotation) as f:
        resultsAnnotation = f.read().splitlines()
    
    missed = 0
    
    total_papers = 0
    
    global_t = 0
    global_d = 0
    global_m = 0
    global_s = 0
    global_td = 0
    global_tdm = 0
    global_tdms = 0
        
    
    for paper in resultsAnnotation:
        
        task_set = set()
        dataset_set = set()
        metric_set = set()
        score_set = set()
        
        t = 0
        d = 0
        m = 0
        s = 0
        td = 0
        tdm = 0
        tdms = 0
        
        list_parsed_pdf = os.listdir(path_to_txt_folder)
        if '.ipynb_checkpoint' in list_parsed_pdf:
            list_parsed_pdf.remove('.ipynb_checkpoint')
        
        pdf_id = '.'.join(paper.split("\t")[0].split('/')[-1].split('.')[:-1])
        
        # Avoid paper that are not parsed
        if pdf_id+file_extention not in list_parsed_pdf:
            continue 
            
        total_contrib = len(paper.split("\t")[-1].split("$"))
        
        with open(f"{path_to_txt_folder}{pdf_id}{file_extention}", errors='replace') as f:
            txt = f.read().splitlines()
            
        file_txt = ' '.join(txt)
        file_txt = re.sub(r'[^\w]', ' ', file_txt)
        
        for contrib in paper.split("\t")[-1].split("$"):
            if len(contrib.split("#")) != 4:
                missed += 1
                continue
#                 ipdb.set_trace()
            task, dataset, metric, score = contrib.split("#")
            task, dataset, metric, score = re.sub(r'[^\w]', ' ', task), re.sub(r'[^\w]', ' ', dataset), re.sub(r'[^\w]', ' ', metric), re.sub(r'[^\w]', ' ', score)
            found_task = re.search(rf"\b{task}\b", file_txt)
            found_dataset = re.search(rf"\b{dataset}\b", file_txt)                  
            found_metric = re.search(rf"\b{metric}\b", file_txt)
            found_score = re.search(rf"\b{score}\b", file_txt)
            
            if found_task and found_dataset and found_metric and found_score:
                tdms += 1
            if found_task and found_dataset and found_metric:
                tdm += 1
            if found_task and found_dataset:
                td += 1          
            if found_task:
                t += 1           
            if found_dataset:
                d += 1             
            if found_metric:
                m += 1
            if found_score:
                s += 1
        
        global_t += t/total_contrib
        global_d += d/total_contrib
        global_m += m/total_contrib
        global_s += s/total_contrib
        global_td += td/total_contrib
        global_tdm += tdm/total_contrib
        global_tdms += tdms/total_contrib
        
#         # Relax the count of found TDN within a contrib
#         global_t += t/(total_contrib/2)
#         global_d += d/(total_contrib/2)
#         global_m += m/(total_contrib/2)
#         global_s += s/(total_contrib/2)
#         global_td += td/(total_contrib/2)
#         global_tdm += tdm/(total_contrib/2)
#         global_tdms += tdms/(total_contrib/2)
        
        total_papers += 1
        
    
    print(f"Pourcentage task {round((global_t/total_papers)*100, 2)} %")
    print(f"Pourcentage dataset {round((global_d/total_papers)*100, 2)} %")
    print(f"Pourcentage metric {round((global_m/total_papers)*100, 2)} %")
    print(f"Pourcentage score {round((global_s/total_papers)*100, 2)} %")
    print(f"Pourcentage task-dataset {round((global_td/total_papers)*100, 2)} %")
    print(f"Pourcentage task-dataset-metric {round((global_tdm/total_papers)*100, 2)} %")
    print(f"Pourcentage task-dataset-metric-score {round((global_tdms/total_papers)*100, 2)} %")
    
    print(f"Total missed data {missed} and Total number of paper {total_papers}")
    return 

In [7]:
get_stats_from_files(path_to_annotation="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/resultsAnnotation.tsv", 
                         file_extention=".txt",
                         path_to_txt_folder="/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_txt/")

Pourcentage task 39.0 %
Pourcentage dataset 48.79 %
Pourcentage metric 33.38 %
Pourcentage score 18.81 %
Pourcentage task-dataset 19.59 %
Pourcentage task-dataset-metric 6.78 %
Pourcentage task-dataset-metric-score 1.44 %
Total missed data 16 and Total number of paper 5193


### From Arxiv source code 

In [8]:
# .tex file 
get_stats_from_files(path_to_annotation="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/resultsAnnotation.tsv",
                          file_extention=".tex", 
                          path_to_txt_folder="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/exp/arxiv_src/")

Pourcentage task 32.84 %
Pourcentage dataset 39.19 %
Pourcentage metric 30.87 %
Pourcentage score 48.0 %
Pourcentage task-dataset 17.1 %
Pourcentage task-dataset-metric 7.2 %
Pourcentage task-dataset-metric-score 5.22 %
Total missed data 18 and Total number of paper 5195


In [9]:
# .txt file converted using pandoc
get_stats_from_files(path_to_annotation="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/resultsAnnotation.tsv",
                          file_extention=".txt", 
                          path_to_txt_folder="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/exp/arxiv_src_txt/")

Pourcentage task 26.92 %
Pourcentage dataset 50.12 %
Pourcentage metric 37.51 %
Pourcentage score 49.79 %
Pourcentage task-dataset 14.01 %
Pourcentage task-dataset-metric 5.71 %
Pourcentage task-dataset-metric-score 3.28 %
Total missed data 10 and Total number of paper 3574


### IBM DocTEA Context

In [32]:
f1_train = "/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/new/60Neg800unk/twofoldwithunk/fold1/train.tsv"
f1_test = "/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/new/60Neg800unk/twofoldwithunk/fold1/test.tsv"
f2_train = "/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/new/60Neg800unk/twofoldwithunk/fold2/train.tsv"
f2_test = "/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/new/60Neg800unk/twofoldwithunk/fold2/test.tsv"

In [33]:
dict_paper_contest = {}
with open(f"{f1_train}") as f:
    txt_f1_train = f.read().splitlines()
with open(f"{f1_test}") as f:
    txt_f1_test = f.read().splitlines()
with open(f"{f2_train}") as f:
    txt_f2_train = f.read().splitlines()
with open(f"{f2_test}") as f:
    txt_f2_test = f.read().splitlines()

In [34]:
len(txt_f1_train)

256008

In [35]:
def fill_dict(cache_dict, list_txt_files):
    for item in list_txt_files:
        id_file = item.split("\t")[1].split(".pdf")[0]
        content = item.split("\t")[-1]
        if id_file not in cache_dict.keys():
            cache_dict[id_file] = content

In [36]:
fill_dict(dict_paper_contest, txt_f1_train)
fill_dict(dict_paper_contest, txt_f1_test)
fill_dict(dict_paper_contest, txt_f2_train)
fill_dict(dict_paper_contest, txt_f2_test)

In [37]:
len(dict_paper_contest)

5193

In [39]:
def get_stats_from_files_with_dict(path_to_annotation="resultsAnnotation.tsv",
                          dict_annotation=dict_paper_contest,
                          path_to_txt_folder="../pdf_txt/"):
    with open(path_to_annotation) as f:
        resultsAnnotation = f.read().splitlines()
    
    missed = 0
    
    total_papers = 0
    
    global_t = 0
    global_d = 0
    global_m = 0
    global_s = 0
    global_td = 0
    global_tdm = 0
    global_tdms = 0
        
    
    for paper in resultsAnnotation:
        
        task_set = set()
        dataset_set = set()
        metric_set = set()
        score_set = set()
        
        t = 0
        d = 0
        m = 0
        s = 0
        td = 0
        tdm = 0
        tdms = 0
        
        list_parsed_pdf = os.listdir(path_to_txt_folder)
        if '.ipynb_checkpoint' in list_parsed_pdf:
            list_parsed_pdf.remove('.ipynb_checkpoint')
        
        pdf_id = '.'.join(paper.split("\t")[0].split('/')[-1].split('.')[:-1])
        
        # Avoid paper that are not parsed
        if pdf_id+".txt" not in list_parsed_pdf or pdf_id not in dict_annotation.keys():
            continue 
        total_contrib = len(paper.split("\t")[-1].split("$"))      
        
        file_txt = dict_paper_contest[pdf_id]
        file_txt = re.sub(r'[^\w]', ' ', file_txt)
        
        for contrib in paper.split("\t")[-1].split("$"):
            if len(contrib.split("#")) != 4:
                missed += 1
                continue
#                 ipdb.set_trace()
            task, dataset, metric, score = contrib.split("#")
            task, dataset, metric, score = re.sub(r'[^\w]', ' ', task), re.sub(r'[^\w]', ' ', dataset), re.sub(r'[^\w]', ' ', metric), re.sub(r'[^\w]', ' ', score)
            found_task = re.search(rf"\b{task}\b", file_txt)
            found_dataset = re.search(rf"\b{dataset}\b", file_txt)                  
            found_metric = re.search(rf"\b{metric}\b", file_txt)
            found_score = re.search(rf"\b{score}\b", file_txt)
            
            if found_task and found_dataset and found_metric and found_score:
                tdms += 1
            if found_task and found_dataset and found_metric:
                tdm += 1
            if found_task and found_dataset:
                td += 1          
            if found_task:
                t += 1           
            if found_dataset:
                d += 1             
            if found_metric:
                m += 1
            if found_score:
                s += 1
            
        global_t += t/total_contrib
        global_d += d/total_contrib
        global_m += m/total_contrib
        global_s += s/total_contrib
        global_td += td/total_contrib
        global_tdm += tdm/total_contrib
        global_tdms += tdms/total_contrib
        
        total_papers += 1
        
    print(f"Pourcentage task {round((global_t/total_papers)*100, 2)}")
    print(f"Pourcentage dataset {round((global_d/total_papers)*100, 2)}")
    print(f"Pourcentage metric {round((global_m/total_papers)*100, 2)}")
    print(f"Pourcentage score {round((global_s/total_papers)*100, 2)}")
    print(f"Pourcentage task-dataset {round((global_td/total_papers)*100, 2)}")
    print(f"Pourcentage task-dataset-metric {round((global_tdm/total_papers)*100, 2)}")
    print(f"Pourcentage task-dataset-metric-score {round((global_tdms/total_papers)*100, 2)}")
    
    return 

In [40]:
get_stats_from_files_with_dict(path_to_annotation="/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/resultsAnnotation.tsv",
                          dict_annotation=dict_paper_contest, 
                          path_to_txt_folder="/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_txt/")

Pourcentage task 29.34
Pourcentage dataset 43.41
Pourcentage metric 30.09
Pourcentage score 6.18
Pourcentage task-dataset 13.41
Pourcentage task-dataset-metric 4.6
Pourcentage task-dataset-metric-score 0.45


# IBM Data eval

In [19]:
f1_IBM_train = "../../ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/train.tsv"
f1_IBM_test = "../../ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/test.tsv"

dict_paper_IBM_contest = {}
with open(f"{f1_IBM_train}") as f:
    txt_f1_IBM_train = f.read().splitlines()
with open(f"{f1_IBM_test}") as f:
    txt_f1_IBM_test = f.read().splitlines()
    
def fill_dict_IBM(cache_dict, list_txt_files):
    for item in list_txt_files:
        id_file = item.split("\t")[1].split(".pdf")[0]
        content = item.split("\t")[-1]
        if id_file not in cache_dict.keys():
            cache_dict[id_file] = content

In [15]:
# Get paper that are in In IBM eval not in Our Eval

In [41]:
len(dict_paper_contest)

5193

In [42]:
fill_dict_IBM(dict_paper_IBM_contest, txt_f1_IBM_train)
fill_dict_IBM(dict_paper_IBM_contest, txt_f1_IBM_test)

In [43]:
len(dict_paper_IBM_contest)

337

In [46]:
missing_paper = set()

for paper in dict_paper_IBM_contest:
    if paper not in dict_paper_contest.keys():
        missing_paper.add(paper)

In [47]:
len(missing_paper)

337

In [32]:
# missing_paper

In [37]:
list(dict_paper_IBM_contest.keys())[:5]

['D16-1036', '1803.09074', '1806.06228', 'P17-1127', '1602.02373']

In [40]:
'1806.06228' in dict_paper_contest.keys()

False

## Creat TDM taxonomy for IBM experiment reproducibility  

In [ ]:
task-dataset-metric-extraction/data/ibm/NLP-TDMS/annotations/resultsAnnotation.tsv

In [48]:
results_Annotation_IBM = "../../ibm/NLP-TDMS/annotations/resultsAnnotation.tsv"

with open(f"{results_Annotation_IBM}") as f:
    list_results_Annotation_IBM = f.read().splitlines()
    
# with open(f"{f1_IBM_test}") as f:
#     txt_f1_IBM_test = f.read().splitlines()
    
# def fill_dict_IBM(cache_dict, list_txt_files):
#     for item in list_txt_files:
#         id_file = item.split("\t")[1].split(".pdf")[0]
#         content = item.split("\t")[-1]
#         if id_file not in cache_dict.keys():
#             cache_dict[id_file] = content

In [63]:
from collections import defaultdict
dict_results_Annotation_IBM = defaultdict(lambda: 0 )

for paper_list in list_results_Annotation_IBM:
    for paper in paper_list.split("$"):
        dict_results_Annotation_IBM[paper.split("\t")[-1].rsplit('#', 1)[0]] +=1

In [64]:
len(dict_results_Annotation_IBM)

171

In [65]:
dict_results_Annotation_IBM

defaultdict(<function __main__.<lambda>()>,
            {'part-of-speech_tagging#UD#Avg Accuracy': 3,
             'summarization#CNN / Daily Mail (Non-anonymized version)#ROUGE-1': 14,
             'summarization#CNN / Daily Mail (Non-anonymized version)#ROUGE-2': 14,
             'summarization#CNN / Daily Mail (Non-anonymized version)#ROUGE-L': 14,
             'summarization#CNN / Daily Mail (Non-anonymized version)#METEOR': 14,
             'summarization#Gigaword#ROUGE-1': 13,
             'summarization#Gigaword#ROUGE-2': 13,
             'summarization#Gigaword#ROUGE-L': 13,
             'summarization#DUC 2004 Task 1#ROUGE-1': 7,
             'summarization#DUC 2004 Task 1#ROUGE-2': 7,
             'summarization#DUC 2004 Task 1#ROUGE-L': 7,
             'summarization#CNN / Daily Mail (Anonymized version)#ROUGE-1': 11,
             'summarization#CNN / Daily Mail (Anonymized version)#ROUGE-2': 11,
             'summarization#CNN / Daily Mail (Anonymized version)#ROUGE-L': 11,

In [67]:
def parse_TDM_taxonomy(TDM_taxonomy):      
    with open("../../ibm/NLP-TDMS/annotations/TDM_taxonomy", "a+", encoding="utf-8") as text_file:
        for key, value in TDM_taxonomy.items():
            text_file.write(key+"\t"+str(value)+"\n")

In [68]:
parse_TDM_taxonomy(dict_results_Annotation_IBM)

## The main data

In [3]:
path_to_dataset_train = "../60Neg800unk/twofoldwithunk/fold1/train.tsv"
path_to_dataset_test = "../60Neg800unk/twofoldwithunk/fold1/test.tsv"

In [4]:
with open(path_to_dataset_train) as f:
    path_to_dataset_train_list = f.read().splitlines()
    
with open(path_to_dataset_test) as f:
    path_to_dataset_test_list = f.read().splitlines()

In [5]:
len(path_to_dataset_train_list)

142396

In [6]:
142396

142396

In [7]:
len(path_to_dataset_test_list)

62450

In [8]:
62450

62450

In [9]:
path_to_dataset_train_list[:2]

["true\t1409.0473v7.pdf\tMachine Translation; IWSLT2015 German-English; BLEU score\tPublished as a conference paper at ICLR 2015 NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE Neural machine translation is a recently proposed approach to machine translation. Unlike the traditional statistical machine translation, the neural machine translation aims at building a single neural network that can be jointly tuned to maximize the translation performance. The models proposed recently for neu-ral machine translation often belong to a family of encoder-decoders and encode a source sentence into a fixed-length vector from which a decoder generates a translation. In this paper, we conjecture that the use of a fixed-length vector is a bottleneck in improving the performance of this basic encoder-decoder architecture , and propose to extend this by allowing a model to automatically (soft-)search for parts of a source sentence that are relevant to predicting a target word, wi

In [31]:
TDM_train = set()
Paper_train = set()
Uniq_task_train = set()
Uniq_dataset_train = set()
Uniq_metric_train = set()

unknown_count_train = 0

for contrib in path_to_dataset_train_list:
    label, title, tdm, _ = contrib.split('\t')
    if label =='true':
        TDM_train.add(tdm)
        Paper_train.add(title)
        if len(tdm.split(';')) == 3:
            t, d, m = tdm.split(';')
            Uniq_task_train.add(t.strip())
            Uniq_dataset_train.add(d.strip())
            Uniq_metric_train.add(m.strip())
        elif(tdm == "unknow"):
            unknown_count_train += 1

In [44]:
len(Paper_train)

1862

In [32]:
len(TDM_train)

1226

In [47]:
len(Uniq_task_train)

189

In [48]:
len(Uniq_dataset_train)

663

In [34]:
len(Uniq_metric_train)

340

In [45]:
unknown_count_train

582

In [49]:
TDM_test = set()
Paper_test = set()
Uniq_task_test = set()
Uniq_dataset_test = set()
Uniq_metric_test = set()

unknown_count_test = 0

for contrib in path_to_dataset_test_list:
    label, title, tdm, _ = contrib.split('\t')
    if label =='true':
        TDM_test.add(tdm)
        Paper_test.add(title)
        if len(tdm.split(';')) == 3:
            t, d, m = tdm.split(';')
            Uniq_task_test.add(t.strip())
            Uniq_dataset_test.add(d.strip())
            Uniq_metric_test.add(m.strip())
        elif(tdm == "unknow"):
            unknown_count_test += 1

In [50]:
len(Paper_test)

805

In [52]:
unknown_count_test

218

In [38]:
len(TDM_test)

1032

In [40]:
len(Uniq_task_test)

155

In [41]:
len(Uniq_dataset_test)

559

In [42]:
len(Uniq_metric_test)

276

In [28]:
'Object Detection; COCO minival; APM'.split(";")[1]

' COCO minival'

In [15]:
len(Paper_train)

1862

In [16]:
TDM_test = set()
Paper_test = set()

for contrib in path_to_dataset_test_list:
    label, title, tdm, _ = contrib.split('\t')
    if label =='true':
        TDM_test.add(tdm)
        Paper_test.add(title)

In [17]:
len(TDM_test)

1032

In [ ]:
805

In [18]:
len(Paper_test)

805

In [20]:
keep= []
for tdd in TDM_test:
    if tdd not in TDM:
        keep.append(tdd)
len(keep)

114